## **Introduction to Data Science - Nhập môn khoa học dữ liệu - CSC14119**
### **HCMUS - Trường Đại học khoa học tự nhiên - Nov 2024.**
### **Đồ án thực hành cuối kì - Handling Real-World Problem.**
#### **Due:** 24/12/2024.
#### **Lớp:** 22_21.
#### **Giảng viên hướng dẫn:** Thầy Lê Ngọc Thành - Thầy Lê Nhựt Nam.
#### **STT nhóm:** 9.
---
### **Data Modeling - 02**
**Problem**: Trong cuộc sống hiện đại, nhiều người thường gặp khó khăn khi quyết định nấu món ăn nào dựa trên các nguyên liệu có sẵn trong nhà bếp. Điều này không chỉ dẫn đến sự lãng phí thực phẩm mà còn khiến quá trình nấu nướng mất thời gian hơn. Vì vậy, cần một giải pháp để giúp người dùng nhanh chóng tìm kiếm và đề xuất các món ăn phù hợp dựa trên danh sách nguyên liệu có sẵn.

**Description**: Mô hình gợi ý món ăn dựa trên nguyên liệu là một hệ thống đề xuất (recommendation system) sử dụng dữ liệu về nguyên liệu và công thức nấu ăn để cung cấp các gợi ý phù hợp cho người dùng. Người dùng sẽ nhập danh sách nguyên liệu hiện có, và mô hình sẽ phân tích dữ liệu để đưa ra các món ăn khả thi, sắp xếp theo mức độ phù hợp. 

**Solution**: Sử dụng mô hình K-Nearest Neighbors để đề xuất.

### 1. Thuật toán KNN

In [1]:
from Libraries_Used import *
from sklearn.neighbors import KNeighborsClassifier

In [2]:
ingredients_df = pd.read_csv("../Assert/ingredients.csv",sep=",")
ingredients_df.head()

,Name of dish,soda,turmeric mixture,herb,yogurt,butter,beef bone,chili lemongrass fish sauce,beef fillet,chipotle smoked pepper powder,...,kiwi,lime juice,tea leaves,white sesame,eel,green onion,ice cubes,pigeon,chinese sausages,white vinegar
0,10 common problems and mistakes when making bread,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11 ways to use leftover egg yolks,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12 types of nuts for baking,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14 ways to make Sponge cake/Gato for birthday ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,14 ways to use leftover egg whites,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Tách nguyên liệu (input) và tên món ăn (label)
ingredients = ingredients_df.iloc[:,1:].astype(bool)
dishes = ingredients_df.iloc[:,0]

# Khởi tạo và huấn luyện mô hình KNN, do dữ liệu đầu vào là mảng chứa {0,1} (chỉ quan tâm có xuất hiện hay không)
# nên ta sẽ sử dụng Jaccard Distance để tính khoảng cách thay vì Euclidean Distance
k = 5
knn = KNeighborsClassifier(n_neighbors=k, metric='jaccard')
knn.fit(ingredients, dishes)

KNeighborsClassifier(metric='jaccard')

In [4]:
def dishesRecommenderKNN(ingredients_input: pd.DataFrame):
    _, indices = knn.kneighbors(ingredients_input)
    predict_dishes = dishes.iloc[indices[0]]

    return predict_dishes

In [20]:
def convertInputToVector(input):
    ingredients_list = ingredients_df.columns[1:]
    input_df = pd.DataFrame([ingredients_list.isin(input)],columns=ingredients_list)

    return input_df

In [6]:
def cookingRecipes(predict):
    recipes_map = {}
    recipes_df = ingredients_df[ingredients_df["Name of dish"].isin(predict)]
    for _, row in recipes_df.iterrows():
        ingredients_list = list(row[row == 1].index)
        recipes_map[row["Name of dish"]] = ingredients_list

    return recipes_map

In [23]:
def printRecipes(recipes_map):
    for key, value in recipes_map.items():
        print(f'Recipe: {key}')
        print(f'Ingredients: ')
        for i in value:
            print(f'- {i}')

In [37]:
input = ['shrimp','shiitake mushrooms','eggs']
input_df = convertInputToVector(input)

predict = dishesRecommenderKNN(input_df)
print(type(predict))
printRecipes(cookingRecipes(predict))

<class 'pandas.core.series.Series'>
Recipe: Shrimp Soup with Autumn Vegetables
Ingredients: 
- corn
- shrimp
- white radish
- shiitake mushrooms
- carrot
- cauliflower
- coriander
- pink salt
- water
Recipe: Shrimp Floss Using Cooking Robot Team Cuisine
Ingredients: 
- lemongrass
- shrimp
- fish sauce
- water
Recipe: Shiitake Mushrooms Stuffed with Baked Eggs
Ingredients: 
- quail eggs
- salt
- shiitake mushrooms
- pepper
Recipe: Baked Chinese Cabbage
Ingredients: 
- oyster sauce
- salt
- shiitake mushrooms
- kale
- pepper


#### 2. Giao diện GUI

In [29]:
import tkinter as tk
from tkinter import messagebox

In [40]:
def recommend():
    global entry, result_label
    ingredients = entry.get()
    
    if not ingredients.strip():
        messagebox.showwarning("Cảnh báo", "Hãy nhập nguyên liệu!")
        return
    
    processed_ingredients = convertInputToVector([ingredients])
    result = dishesRecommenderKNN(processed_ingredients).to_list()

    if result:
        result_label.config(text="\n".join(result))
    else:
        result_label.config(text="Không tìm thấy món ăn phù hợp!")

In [42]:
root = tk.Tk()
root.title("Hệ thống gợi ý món ăn")

# Label
label = tk.Label(root, text="Nhập nguyên liệu (phân cách bởi dấu phẩy):")
label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

# Entry để nhập nguyên liệu
entry = tk.Entry(root, width=50)
entry.grid(row=1, column=0, padx=10, pady=10)

# Button để gợi ý món ăn
recommend_button = tk.Button(root, text="Gợi ý món ăn", command=recommend)
recommend_button.grid(row=2, column=0, padx=10, pady=10)

# Label để hiển thị kết quả
result_label = tk.Label(root, text="", justify="left", anchor="w", bg="white", width=50, height=10, relief="solid")
result_label.grid(row=3, column=0, padx=10, pady=10)

root.mainloop()